In [1]:
import torch
import clip
from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
device = "cuda:3" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("rn50/rn50.pt", device=device)

In [3]:
import os
try:
    os.mkdir("rn50")
except:
    pass
torch.save(model.state_dict(),"rn50/rn50.pt")

In [4]:
import json
# 从txt文件中读取类名
classes_file = "./data/classes.txt"
class_id={}
with open(classes_file, "r") as f:
    classes = [line.strip() for line in f.readlines()]
for i,c in enumerate(classes):
    class_id[c]=i
train_data=json.load(open("data/captions.json"))
imgs_pth=[ele['pth'] for ele in train_data]
labels=[class_id[ele['caption']] for ele in train_data]

In [5]:
import torch.optim as optim
import torch.nn.functional as F
criterion = F.cross_entropy
batch_size = 64

In [6]:
datasets = []

# for i in tqdm(range(0, len(train_data), batch_size)):
#     img_files = imgs_pth[i:i + batch_size]
#     # print("data/dataset_A/" + img_files[0].split("/")[-1])
#     batch_images = [preprocess(Image.open("data/dataset_A/" + file.split("/")[-1])) for file in img_files]
#     img_tensors = torch.stack(batch_images)
#     label_tensors = torch.tensor(labels[i:i + batch_size])
#     datasets.append([img_tensors,label_tensors])

In [12]:
import pickle
# pickle.dump(datasets,open("datasets_train.pkl","wb"))
datasets=pickle.load(open("datasets_train.pkl","rb"))

In [8]:
with torch.no_grad():
    text = clip.tokenize(classes).to(device)
    text_features = model.encode_text(text)
    text_features /= text_features.norm(dim=-1, keepdim=True)

In [9]:
gap=10
lr=2e-5
tot_loss=0
for i,(imgs,labels) in enumerate(tqdm(datasets)):
    imgs = imgs.to(device)
    labels=labels.to(device)
    image_features = model.encode_image(imgs)
    image_features /= image_features.norm(dim=-1, keepdim=True).detach()
    pred = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    # print(pred[0].max()/pred[0].sum(),labels[0])
    loss=criterion(pred,labels)
    tot_loss+=loss
    loss.backward()
    with torch.no_grad():
        for param in model.visual.layer4.parameters():
            param -= lr * param.grad
    model.zero_grad()
    # if(i%gap==0):
    #     print(i,loss.item())
print(tot_loss/len(datasets))

  0%|          | 0/114 [00:00<?, ?it/s]/data/wangjieyang/anaconda3/envs/vit/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 114/114 [00:39<00:00,  2.87it/s]

tensor(3.5645, device='cuda:3', dtype=torch.float16, grad_fn=<DivBackward0>)


In [10]:
torch.save(model.state_dict(),"rn50/test.pt")